In [18]:
import os
from langchain.docstore.document import Document

os.environ['OPENAI_API_KEY'] = "API KEY HERE"

class MarkdownLoader:
    def __init__(self, directory):
        self.directory = directory

    def load(self):
        documents = []
        for root, _, files in os.walk(self.directory):
            for file in files:
                if file.endswith('.md'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as f:
                        content = f.read()
                        metadata = {'source': file_path}
                        documents.append(Document(page_content=content, metadata=metadata))
        return documents

In [22]:
# Load your documents
loader = MarkdownLoader("docs")
docs = loader.load()
docs

[Document(page_content='---\nid: introduction\ntitle: Introduction\n---\n\n**Polywrap** is a framework that enables easy integration of Web3 protocols into any application. It makes it possible for applications on any platform, written in any language, to read and write data to Web3 protocols.\n\n:::tip\nHave questions or want to get involved? Join our [Discord](https://discord.polywrap.io/) or [open an issue](https://github.com/polywrap/toolchain/issues/new/choose) on our GitHub repo.\n:::\n\n## The problem with current Web3 SDKs\n\nWe see traditional SDKs as the primary bottleneck to web3\'s potential for composable, cross-platform dApps.\n\nWeb3 promises a new depth of composability in software development.  Any application can interact with smart contract protocols like Uniswap or Aave.  For web3 developers, software development kits (SDKs) simplify this integration process.\n\nHowever, traditional SDKs come with a number of issues for web3 developers, namely they are:\n\n- **diffi

In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

Created a chunk of size 2012, which is longer than the specified 1000
Created a chunk of size 1583, which is longer than the specified 1000
Using embedded DuckDB without persistence: data will be transient


In [24]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [25]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [26]:
query = "How can i make a polywrap plugin?"
result = qa({"question": query})

In [27]:
result

{'question': 'How can i make a polywrap plugin?',
 'chat_history': [HumanMessage(content='How can i make a polywrap plugin?', additional_kwargs={}, example=False),
  AIMessage(content=" You can make a Polywrap plugin by following the [Plugin an existing SDK](/tutorials/create-plugin-wrappers/create-js-plugin) guide. You can also use the `npx polywrap create plugin typescript <project-name>` command to spin up a project folder for your plugin. Once complete, you'll need to install the project's dependencies with `nvm install && nvm use` and `yarn`, and then configure the plugin as needed.", additional_kwargs={}, example=False)],
 'answer': " You can make a Polywrap plugin by following the [Plugin an existing SDK](/tutorials/create-plugin-wrappers/create-js-plugin) guide. You can also use the `npx polywrap create plugin typescript <project-name>` command to spin up a project folder for your plugin. Once complete, you'll need to install the project's dependencies with `nvm install && nvm 